In [1]:
import xml.etree.ElementTree as et
import pandas as pd
import os
import sqlite3
import csv
import glob

<h3> The next cells are for converting the downloaded XML file to CSV. The given file to me was corrupted.</h3>

In [2]:
cols = ["Id", "Reputation", "CreationDate", "DisplayName", 
        "LastAccessDate","WebsiteUrl", "AboutMe", "Location",
        "Views", "UpVotes", "DownVotes", "AccountId"]
rows = []
  
# Parsing the XML file
path = os.getcwd()
dataPath = path + '\datascience.stackexchange.com\\'
xmlName = 'Users'
dataPath = dataPath + xmlName +'.xml'
savePath = path + '\csv\\' + xmlName + '.csv'


xmlparse = et.parse(dataPath)
root = xmlparse.getroot()

for node in root:
    id_ = node.attrib.get("Id")
    reputation = node.attrib.get("Reputation")
    creationDate = node.attrib.get("CreationDate")
    displayName = node.attrib.get("DisplayName")
    lastAccessDate = node.attrib.get("LastAccessDate")
    websiteUrl = node.attrib.get("WebsiteUrl")
    aboutMe = node.attrib.get("AboutMe")
    location = node.attrib.get("Location")
    views = node.attrib.get("Views")
    upVotes = node.attrib.get("UpVotes")
    downVotes = node.attrib.get("DownVotes")
    accountId = node.attrib.get("AccountId")
    
    

    rows.append({"Id": id_,
                "Reputation": reputation,
                "CreationDate": creationDate,
                "DisplayName": displayName,
                 "LastAccessDate":lastAccessDate, 
                 "WebsiteUrl":websiteUrl,
                 "AboutMe":aboutMe,
                 "Location":location,
                 "Views":views,
                 "UpVotes":upVotes,
                 "DownVotes":downVotes,
                 "AccountId":accountId            
                })
  
df = pd.DataFrame(rows, columns=cols,)
df.to_csv(savePath,index=False)

In [3]:
cols = ["Id", "PostId", "VoteTypeId", "CreationDate"]
rows = []
  
# Parsing the XML file
dataPath = path + '\datascience.stackexchange.com\\'
xmlName = 'Votes'
dataPath = dataPath + xmlName +'.xml'
savePath = path + '\csv\\' + xmlName + '.csv'


xmlparse = et.parse(dataPath)
root = xmlparse.getroot()

for node in root:
    id_ = node.attrib.get("Id")
    postId = node.attrib.get("PostId")
    voteTypeId = node.attrib.get("VoteTypeId")
    creationDate = node.attrib.get("CreationDate")
    

    rows.append({"Id": id_,
                "PostId": postId,
                "VoteTypeId": voteTypeId,
                "CreationDate": creationDate          
                })
  
df = pd.DataFrame(rows, columns=cols,)
df.to_csv(savePath,index=False)


In [4]:
cols = ["Id", "TagName", "Count", "ExcerptPostId", "WikiPostId"]
rows = []
  
# Parsing the XML file
dataPath = path + '\datascience.stackexchange.com\\'
xmlName = 'Tags'
dataPath = dataPath + xmlName +'.xml'
savePath = path + '\csv\\' + xmlName + '.csv'

xmlparse = et.parse(dataPath)
root = xmlparse.getroot()

for node in root:
    id_ = node.attrib.get("Id")
    tagName = node.attrib.get("TagName")
    count = node.attrib.get("Count")
    excerptPostId = node.attrib.get("ExcerptPostId")
    wikiPostId = node.attrib.get("WikiPostId")

    rows.append({"Id": id_,
                "TagName": tagName,
                "Count": count,
                "ExcerptPostId": excerptPostId,
                "WikiPostId": wikiPostId,
                })
  
df = pd.DataFrame(rows, columns=cols,)
df.to_csv(savePath,index=False)


<h3> For storing all the CSVs as table into one database (.db). </h3>

In [5]:
def do_directory(dirname, db):
    for filename in glob.glob(os.path.join(dirname, '*.csv')):
        do_file(filename, db)

def do_file(filename, db):
    with open(filename, encoding='utf8') as f:
        with db:
            data = csv.DictReader(f)
            cols = data.fieldnames
            table=os.path.splitext(os.path.basename(filename))[0]

            sql = 'drop table if exists "{}"'.format(table)
            db.execute(sql)

            sql = 'create table "{table}" ( {cols} )'.format(
                table=table,
                cols = ','.join('"{}" "{}"'.format(col, pd.read_csv(f.name)[col].dtype) for col in cols))
                #cols=','.join('"{}"'.format(col) for col in cols))
            db.execute(sql)

            sql = 'insert into "{table}" values ( {vals} )'.format(
                table=table,
                vals=','.join('?' for col in cols))
            db.executemany(sql, (list(map(row.get, cols)) for row in data))
            
def read_all_tables(connection):
    files_dict = {}
    db_tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table'", connection)
    for table in db_tables['name']:
        files_dict[table] = read_table(table, connection)
    return files_dict
def read_table(table_name, connection):
     return pd.read_sql_query("SELECT * from {}".format(table_name), connection)

conn = sqlite3.connect('datascience-stackexchange.db')
do_directory('.\csv', conn)
files_df = read_all_tables(conn)